# Preparando o ambiente

In [1]:
from modules.utils import prepare_data_filesystem

prepare_data_filesystem()

[06/04/2024 00:36:09] Tentando criar o diretório data/raw.
[06/04/2024 00:36:09] O diretório data/raw já existe, nada será feito.
[06/04/2024 00:36:09] Tentando criar o diretório data/processed.
[06/04/2024 00:36:09] O diretório data/processed já existe, nada será feito.


# Programando as execuções

## Landing Step

In [5]:
from modules.etl import landing_step
from datetime import datetime, timedelta

for i in range(0,50):
    inicio = (datetime.now() - timedelta(days=i)).replace(hour=0,minute=0,second=0).strftime(format="%Y-%m-%dT%H:%M:%S")
    fim = (datetime.now() - timedelta(days=i)).replace(hour=23,minute=59,second=59).strftime(format="%Y-%m-%dT%H:%M:%S")
    landing_step(query="dengue",languague="pt",inicio=inicio,fim=fim)

[06/04/2024 00:45:53] FIST STEP (FOR EACH 1 HOUR)
[06/04/2024 00:45:53] Extracting...
[06/04/2024 00:45:53] Construindo os dados a serem enviados na requisição
[06/04/2024 00:45:53] Enviando a requisição
[06/04/2024 00:45:54] Requisição feita com sucesso!!
[06/04/2024 00:45:54] Verificando se é necessárioa mais de uma página
[06/04/2024 00:45:54] Mais de uma página não é necessaŕio! Só um json será salvo!
[06/04/2024 00:45:54] Não há novos artigos no periodo de 2024-04-06T00:00:00 a 2024-04-06T23:59:59
[06/04/2024 00:45:54] FIST STEP (FOR EACH 1 HOUR)
[06/04/2024 00:45:54] Extracting...
[06/04/2024 00:45:54] Construindo os dados a serem enviados na requisição
[06/04/2024 00:45:54] Enviando a requisição
[06/04/2024 00:45:54] Requisição feita com sucesso!!
[06/04/2024 00:45:54] Verificando se é necessárioa mais de uma página
[06/04/2024 00:45:54] Mais de uma página não é necessaŕio! Só um json será salvo!
[06/04/2024 00:45:54] Saving response in data/raw
[06/04/2024 00:45:54] FIST STEP (

## Bronze Step

In [3]:
from modules.etl import bronze_step

bronze_step()

[06/04/2024 00:45:08] Listando os jsons que estão no diretório data/raw
[06/04/2024 00:45:08] Verificando se o diretório data/aw está vazio
[06/04/2024 00:45:08] Criando uma lsita de objetos json
[06/04/2024 00:45:08] Transformando-os em dicionários Python
[06/04/2024 00:45:08] Separando somente a key articles
[06/04/2024 00:45:08] Transformando em dataframes
[06/04/2024 00:45:08] COncatenando os dataframes
[06/04/2024 00:45:08] Inserindo a colunad 'load_date'
[06/04/2024 00:45:08] Verificando se já tem um arquivo bronze


## Silver Step

In [42]:
import pandas as pd

df = pd.read_parquet("data/processed/bronze.parquet")

# Tratando os tipos de variáveis
df['publishedAt'] = pd.to_datetime(df['publishedAt']) # datetime
df['load_date'] = pd.to_datetime(df['load_date']) # datetime
df['source'] = df['source'].apply(lambda x:x.get("name").lower()) # Trabalhando na coluna source
df['source'] = df['source'].astype("category") # Categorico
df = df[df['content']!="[Removed]"] # Retirando artigos que tiveram o conteúdo removido
df.drop_duplicates(inplace=True) # Retirando registros duplicados

# Respondendo as perguntas

## Quantidade de noticias por ano, mês e dia de publicação